<a href="https://colab.research.google.com/github/AntoninCamus/AI_ChatBot/blob/master/text_classification_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#my first notebook

In [2]:
#mout my drive, where i put all the data
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
#dataset from https://archive.ics.uci.edu/ml/datasets/Sentiment+Labelled+Sentences
#includes our set and 2 others (imdb, yealp)
#reach my drive and check if it is there: 
!ls "/content/gdrive/My Drive/Colab Notebooks/dataset/sentiment labelled sentences/sentiment labelled sentences"

amazon_cells_labelled.txt  imdb_labelled.txt  readme.txt  yelp_labelled.txt


In [0]:
#I will now try to follow this tutorial:
#https://medium.com/saarthi-ai/sentence-classification-using-convolutional-neural-networks-ddad72c7048c

In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer                    
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras import layers

#read the data in a pandas dataframe
path = "/content/gdrive/My Drive/Colab Notebooks/dataset/sentiment labelled sentences/sentiment labelled sentences/"
filepath_dict = {'yelp': 'yelp_labelled.txt' ,'amazon': 'amazon_cells_labelled.txt','imdb': 'imdb_labelled.txt'}
df_list = []
for source, filepath in filepath_dict.items():
  filepath = path + filepath
  df = pd.read_csv(filepath, names=['sentence', 'label'], sep='\t')
  #Add another column filled with the source name
  df['source'] = source 
  df_list.append(df)
df = pd.concat(df_list)

Using TensorFlow backend.


In [6]:
print(df.head())
print(df.tail())

                                            sentence  label source
0                           Wow... Loved this place.      1   yelp
1                                 Crust is not good.      0   yelp
2          Not tasty and the texture was just nasty.      0   yelp
3  Stopped by during the late May bank holiday of...      1   yelp
4  The selection on the menu was great and so wer...      1   yelp
                                              sentence  label source
743  I just got bored watching Jessice Lange take h...      0   imdb
744  Unfortunately, any virtue in this film's produ...      0   imdb
745                   In a word, it is embarrassing.        0   imdb
746                               Exceptionally bad!        0   imdb
747  All in all its an insult to one's intelligence...      0   imdb


In [7]:
df = df #use the whole data
# df = df[df['source'] == 'amazon'] #use only amazon

sentences = df['sentence'].values #extract sentences
y = df['label'].values #extract labels

#split datasets intto training and testing
sentences_train, sentences_test, y_train, y_test = train_test_split(sentences, y, test_size=0.25, random_state=1000)

#turn words into sequence of ints
#only the most common num_words-1 words will be kept.
tokenizer = Tokenizer(num_words=5000) 

#learn word_counts, word_docs, word_index (which we will use), document_count:
tokenizer.fit_on_texts(sentences_train) 

print('before vectorization:', sentences_train[0])
X_train = tokenizer.texts_to_sequences(sentences_train)
X_test = tokenizer.texts_to_sequences(sentences_test)
print('after vectorization:', X_train[0])
# Adding 1 because of  reserved 0 index
vocab_size = len(tokenizer.word_index) + 1                          

maxlen = 100
#pad them into the same lengths
X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)
print('after padding:', X_train[0])

before vectorization: I'd rather eat airline food, seriously.
after vectorization: [278, 295, 212, 1907, 39, 349]
after padding: [ 278  295  212 1907   39  349    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]


In [8]:
embedding_dim = 100 #from tutorial, why?

model = Sequential()
model.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen))
model.add(layers.Conv1D(128, 5, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',loss='binary_crossentropy', metrics=['accuracy'])






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


## Comments on the loss function: 
We use binary_crossentropy because our classifier is binary.
([explained here](https://towardsdatascience.com/understanding-binary-cross-entropy-log-loss-a-visual-explanation-a3ac6025181a))

\begin{equation}
L(y,\hat{y}) = -\frac{1}{N}\sum_{i=0}^{N}(y\cdot \log(\hat{y}_i)+(1-y)\cdot\log(1-\hat{y}_i)
\end{equation}

![image:](https://ml-cheatsheet.readthedocs.io/en/latest/_images/cross_entropy.png)



In [9]:
#train
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), batch_size=10)




Train on 2061 samples, validate on 687 samples
Epoch 1/10





2061/2061 [==============================] - 5s 3ms/step - loss: 0.6452 - acc: 0.6157 - val_loss: 0.4662 - val_acc: 0.8035
Epoch 2/10
2061/2061 [==============================] - 4s 2ms/step - loss: 0.3216 - acc: 0.8826 - val_loss: 0.3747 - val_acc: 0.8457
Epoch 3/10
2061/2061 [==============================] - 4s 2ms/step - loss: 0.1012 - acc: 0.9709 - val_loss: 0.4086 - val_acc: 0.8472
Epoch 4/10
2061/2061 [==============================] - 4s 2ms/step - loss: 0.0289 - acc: 0.9937 - val_loss: 0.4751 - val_acc: 0.8486
Epoch 5/10
2061/2061 [==============================] - 4s 2ms/step - loss: 0.0112 - acc: 0.9981 - val_loss: 0.5240 - val_acc: 0.8515
Epoch 6/10
2061/2061 [==============================] - 4s 2ms/step - loss: 0.0046 - acc: 0.9995 - val_loss: 0.5714 - val_acc: 0.8443
Epoch 7/10
2061/2061 [==============================] - 4s 2ms/step - loss: 0.0018 - acc: 1.0000 - val_loss: 0.5898 - val_acc: 0.8472
Epoch 8